In [120]:
import pandas as pd

In [121]:
min30 = pd.read_csv('XAUUSD_30M.csv')
hourly = pd.read_csv('XAUUSD_1H.csv')
hourly4 = pd.read_csv('XAUUSD_4H.csv')
daily = pd.read_csv('XAUUSD_1D.csv')
weekly = pd.read_csv('XAUUSD_1W.csv')

In [122]:
hourly.columns

Index(['time', 'open', 'high', 'low', 'close', 'tick_volume'], dtype='object')

In [123]:
def process_loaded_data(df):
    # Rename 'time' to 'datetime'
    if 'datetime' not in df.columns and 'time' in df.columns:
        df.rename(columns={'time': 'datetime'}, inplace=True)

    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)

    df['avgPrice'] = (df['open'] + df['close']) / 2
    df['gradient'] = df['avgPrice'].diff()
    df['signal_label'] = df['gradient'].apply(
        lambda x: 'Bullish' if x > 0 else ('Bearish' if x < 0 else 'Neutral')
    )

    df = df.reset_index()
    df['signal_with_time'] = df.apply(
        lambda row: f"{row['signal_label']} ({row['datetime']})", axis=1
    )
    return df

min30 = process_loaded_data(min30)
hourly = process_loaded_data(hourly)
hourly4 = process_loaded_data(hourly4)
daily = process_loaded_data(daily)
weekly = process_loaded_data(weekly)


In [124]:
hourly.head()

,datetime,open,high,low,close,tick_volume,avgPrice,gradient,signal_label,signal_with_time
0,2018-01-02 01:00:00,1302.96,1307.68,1302.26,1306.79,4822,1304.875,NaN,Neutral,Neutral (2018-01-02 01:00:00)
1,2018-01-02 02:00:00,1306.79,1307.30,1305.27,1306.34,4900,1306.565,1.69,Bullish,Bullish (2018-01-02 02:00:00)
2,2018-01-02 03:00:00,1306.33,1307.02,1304.23,1306.20,7842,1306.265,-0.30,Bearish,Bearish (2018-01-02 03:00:00)
3,2018-01-02 04:00:00,1306.17,1306.83,1305.52,1306.00,4655,1306.085,-0.18,Bearish,Bearish (2018-01-02 04:00:00)
4,2018-01-02 05:00:00,1306.00,1307.48,1305.44,1306.47,4536,1306.235,0.15,Bullish,Bullish (2018-01-02 05:00:00)


In [125]:
weekly.head()

,datetime,open,high,low,close,tick_volume,avgPrice,gradient,signal_label,signal_with_time
0,2018-01-07,1320.79,1339.39,1308.25,1337.74,1145869,1329.265,NaN,Neutral,Neutral (2018-01-07 00:00:00)
1,2018-01-14,1336.27,1344.76,1324.21,1331.61,1220963,1333.940,4.675,Bullish,Bullish (2018-01-14 00:00:00)
2,2018-01-21,1334.72,1366.08,1328.41,1350.24,1421705,1342.480,8.540,Bullish,Bullish (2018-01-21 00:00:00)
3,2018-01-28,1349.68,1352.41,1327.48,1332.85,1538868,1341.265,-1.215,Bearish,Bearish (2018-01-28 00:00:00)
4,2018-02-04,1331.56,1346.12,1307.08,1315.97,2226743,1323.765,-17.500,Bearish,Bearish (2018-02-04 00:00:00)


In [126]:
min30.to_csv('XAUUSD_30M_markettrend.csv', index=False)
hourly.to_csv('XAUUSD_1H_markettrend.csv', index=False)
hourly4.to_csv('XAUUSD_4H_markettrend.csv', index=False)
daily.to_csv('XAUUSD_1D_markettrend.csv', index=False)
weekly.to_csv('XAUUSD_1W_markettrend.csv', index=False)

In [127]:
# Ensure 'datetime' column exists and is datetime type
daily['datetime'] = pd.to_datetime(daily['datetime'])

# Set 'datetime' as the index
daily = daily.set_index('datetime')

# Resample monthly with month start
monthly = daily.resample('MS').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
}).dropna()

# No need to assign monthly['datetime'] = monthly.index here!

# Calculate avgPrice and signals
monthly['avgPrice'] = (monthly['open'] + monthly['close']) / 2
monthly['gradient'] = monthly['avgPrice'].diff()
monthly['signal_label'] = monthly['gradient'].apply(
    lambda x: 'Bullish' if x > 0 else ('Bearish' if x < 0 else 'Neutral')
)

# Reset index, datetime will become a column automatically
monthly = monthly.reset_index()

monthly['signal_with_time'] = monthly.apply(
    lambda row: f"{row['signal_label']} ({row['datetime']})", axis=1
)


In [128]:
monthly.head()

,datetime,open,high,low,close,avgPrice,gradient,signal_label,signal_with_time
0,2018-01-01,1302.96,1366.08,1302.26,1345.04,1324.000,NaN,Neutral,Neutral (2018-01-01 00:00:00)
1,2018-02-01,1344.73,1361.76,1307.08,1318.07,1331.400,7.400,Bullish,Bullish (2018-02-01 00:00:00)
2,2018-03-01,1318.00,1356.86,1302.82,1325.15,1321.575,-9.825,Bearish,Bearish (2018-03-01 00:00:00)
3,2018-04-01,1325.50,1365.29,1310.24,1315.27,1320.385,-1.190,Bearish,Bearish (2018-04-01 00:00:00)
4,2018-05-01,1314.72,1325.98,1282.10,1298.25,1306.485,-13.900,Bearish,Bearish (2018-05-01 00:00:00)


In [129]:

monthly.to_csv('XAUUSD_1M_markettrend.csv', index=False)